##### 1) Conectando ao MySQL

In [31]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
import mysql.connector

connection = mysql.connector.connect(
    host = os.getenv("DB_HOST"),
    user = os.getenv("DB_USERNAME"),
    password = os.getenv("DB_PASSWORD")
)

print(connection)

cursor = connection.cursor()

##### 2) Criando o banco de dados

In [2]:
cursor.execute('CREATE DATABASE IF NOT EXISTS db_produtos;')

In [3]:
cursor.execute('SHOW DATABASES;')

for db in cursor:
    print(db)

('db_produtos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


##### 3) Criando a tabela

In [4]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS db_produtos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               PRIMARY KEY (id)
    );
""")

In [5]:
cursor.execute('USE db_produtos;')

In [ ]:
cursor.execute('SHOW TABLES;')
for tb in cursor:
    print(tb)

##### 4) Importando os dados e salvando na tabela

In [7]:
import pandas as pd

df_livros = pd.read_csv('/home/pedro/documents/pipeline-mongodb-mysql/data/tabela_livros.csv')
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,67f03c632ee2b00ba9e60a47,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,67f03c632ee2b00ba9e60a48,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,67f03c632ee2b00ba9e60a81,Dashboards com Power BI,livros,40.98,0.000000,2023-02-01,Beatriz Moraes,RJ,5,cartao_credito,1,-22.25,-42.66
3,67f03c632ee2b00ba9e60af1,Dashboards com Power BI,livros,46.87,0.000000,2021-07-01,Beatriz Moraes,RS,5,cartao_credito,8,-30.17,-53.50
4,67f03c632ee2b00ba9e60b58,Dashboards com Power BI,livros,54.59,3.189879,2022-10-01,Juliana Costa,SP,3,cartao_credito,8,-22.19,-48.79


In [28]:
df_livros.shape

(742, 13)

In [ ]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]

In [13]:
sql = "INSERT INTO db_produtos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados)
connection.commit()

In [16]:
print(cursor.rowcount, "linhas")

742 linhas


##### 5) Desafio, produtos 2021 em diante

In [19]:
df_produtos2021 = pd.read_csv('/home/pedro/documents/pipeline-mongodb-mysql/data/tabela_produtos_2021_em_diante.csv')

In [20]:
lista_dados = [tuple(row) for i, row in df_produtos2021.iterrows()]

In [21]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS db_produtos.tb_produtos2021(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               PRIMARY KEY (id)
    );
""")

In [25]:
sql = "INSERT INTO db_produtos.tb_produtos2021 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados)
connection.commit()

In [26]:
df_produtos2021.shape

(6574, 13)

In [27]:
print(cursor.rowcount, "linhas")

6574 linhas


##### 6) Visualizando os dados

In [29]:
cursor.execute('SELECT * FROM db_produtos.tb_livros;')

for row in cursor:
    print(row)

('67f03c632ee2b00ba9e60a47', 'Modelagem preditiva', 'livros', 92.45, 5.61, datetime.date(2020, 1, 1), 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('67f03c632ee2b00ba9e60a48', 'Iniciando em programação', 'livros', 43.84, 0.0, datetime.date(2020, 1, 1), 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('67f03c632ee2b00ba9e60a5a', 'Iniciando em programação', 'livros', 63.25, 3.89, datetime.date(2022, 1, 1), 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('67f03c632ee2b00ba9e60a68', 'Ciência de dados com python', 'livros', 86.13, 5.27, datetime.date(2021, 2, 1), 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('67f03c632ee2b00ba9e60a6a', 'Ciência de dados com python', 'livros', 72.75, 1.46, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('67f03c632ee2b00ba9e60a6d', 'Iniciando em programação', 'livros', 61.73, 0.0, datetime.date(2021, 2, 1), 'Beatriz Moraes', 'SP', 4, 'cartao_credi

##### 7) Fechando a conexão

In [30]:
cursor.close()
connection.close()